In [9]:
import numpy as np
import pandas as pd
from ete3 import Tree, TreeStyle, TextFace, add_face_to_node# Just a library that helps draw trees

In [2]:
#opening the file that contains the data set and removing the missing values
file = open("breast-cancer.data", "r")
file_data = file.readlines()
data = []
for line in file_data:
    if '?' in line:    #Excluding the missing values in the data set
        continue
    data.append(line.strip("\n").split(','))

In [23]:
features = np.array(['age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 
        'deg-malig', 'breast', 'breast-quad', 'irradiat'])
data = np.array(data)
np.random.shuffle(data)
train_data = data[:221]
validation_data = data[221:260]
test_data = data[260:]

In [24]:
# Write a function which calculates the entropy of a set of data (in general we will only pass catergory labels into the function)
# HINT: look at np.unique, note unique returns in ascending order or alphabetically
def calc_entropy(data):
    # TODO: Write the function to calculate entropy. The function must return the entropy value and an array of the unique values in the data
    unique, prob = np.unique(data,return_counts=True)
    data_entropy = 0
    total = len(data)
    for i in range(0,len(prob)):
        data_entropy = data_entropy + (prob[i]/total)*np.log2(prob[i]/total)
    data_entropy =-data_entropy 
    return([data_entropy, unique]) # returns the entropy of the data as well as the set of unique value for the data

In [25]:
class tree_node:
    def __init__(self, data, labels, diagram_node, available_features):
        self.data = data # holds the data which the tree node will use to find the best feature
        self.labels = labels # holds the corresponding labels for each data point
        self.diagram_node = diagram_node # Used for creating the tree diagram
        self.available_features = available_features # Holds the list of names for the remaining features available at a tree node
        self.feature_index = None # Holds the index of the feature in the list of features which provides the most information gain
        self.feature_name = None # Holds the name of the feature which provides the most information gain
        self.node_values = None # Holds the unique values of the feature which provides the most information gain
        self.is_leaf = False # Reflects if the node is a leaf node
        self.class_value = None # Is set to True or False if a node classifies a data point (it is a leaf node)
        self.children = None # Array to hold the children node of this node
        self.node_entropy = calc_entropy(labels)[0] # Entropy value of the data entering the node 
                                    #(before being split by the feature which provides the most information gain)
        
        # Three cases to consider when adding a node to the tree.
        # 1. The node entropy is not 0, so the node still needs to split the data further, and there is still
        #    an unused feature to split the data with
        # 2. If the entropy is not 0 but we have already used all features to split the data
        # 3. The entropy of the data coming into the node is 0 and so the node must just pick a classification
        #    (ie: True or False)
        # It may be helpful to first complete the class functions below to see how they work
        if not (self.node_entropy == 0.0 or self.node_entropy == -0.0 or self.data.shape[1] == 0):
            # TODO: Init an empty array for self.children, then find the feature for this node to split the data,
            #       then descend the tree (add children to the current node)
            self.children = np.array([]);
            self.find_feature()
            self.descend_tree()
           
        elif self.data.shape[1] == 0:
            # TODO: Update self.is_leaf, get the unique values and their counts of the labels for this node's data,
            #       then find the index of the unique value with the largest count
            self.is_leaf = True
            unique, counts = np.unique(self.labels,return_counts=True)
            majority_class = np.argmax(counts)
            self.class_value = unique[majority_class]
            self.feature_name = str(self.class_value)
            self.diagram_node.name = self.feature_name
        else:
            self.is_leaf = True
            self.class_value = labels[0]
            self.feature_name = str(labels[0])
            self.diagram_node.name = self.feature_name

    # This function is used to calculate the information gain of each feature and pick the best feature
    def find_feature(self):
        #print("Finding feature for new node")
        feature_entropies = np.zeros(len(self.available_features ))# TODO # initialize the entropy of each feature to 0
        info_gains = np.where(feature_entropies==0,self.node_entropy,feature_entropies)# TODO # init the info gain for each feature to equal the
                                     # entropy of the data coming into the done (ie: H(D) in the formula above)
        for i in range(self.data.shape[1]): # For each feature
            print("Working on feature: ", i)
            feature_entropies[i], _ = calc_entropy(self.labels) # TODO # calculate the entropy of the data into the node
            feature_sub_classes = np.unique(self.data[:,i]) # TODO # find the unique values for the feature we are calculation info gain for
            for j in feature_sub_classes: # TODO # for each unique value of the feature
                indeces = np.where(self.data[:,i]==j) # TODO # find the data points where this feature value occurs
                sub_clas_data = []
                for ind in indeces:
                    sub_clas_data.append(self.labels[ind])
                data_ratio =len(sub_clas_data)/len(self.data[:,i]) # TODO # calc how much of the total data has this feature value
                sub_clas_entropy = calc_entropy(sub_clas_data)[0]# TODO # calc entropy for the subset of data with the feature value
                info_gains[i] -=data_ratio*sub_clas_entropy # TODO # update the information gain
            print(info_gains)
        chosen_feature = np.argmax(info_gains)                       # TODO # choose feature which gives mac info gain
        self.feature_index = chosen_feature # update features of the node class
        self.feature_name = self.available_features[self.feature_index]
        self.diagram_node.name = self.feature_name
        print("Found feature: ", self.feature_name)

    # This function is used to add nodes to the tree once the best feature to split the data is found
    def descend_tree(self):
        print("Descending tree with node entropy value: ", self.node_entropy)
        unique, counts = np.unique(self.data[:,self.feature_index],return_counts=True) # TODO # Find the unique values of the chosen feature
        self.node_values = unique # Update class values which holds the values of the feature it uses
        for i in unique: # TODO # For each unqiue value the chosen feature can take
            data_for_feature_value = np.where(self.data[:,self.feature_index]==i) # TODO # Find data where unique value for the feature occurs
            remaining_features = np.arange(self.data.shape[1])!=self.feature_index # This just drops the chosen feature from the list of unused feature names (NOTE useful for inference below)
            new_child_diagram_node = self.diagram_node.add_child(name="Temp") # used for making the tree diagram
            
            # For each unique value of the chosen feature we add a new node. Some points to note
            # First we only use the data which ass the unique value we are looking for for the feature 
            # this is found in the "data_for_feature_value" variable above
            # Secondly we remove the feature we used to split the data, the unused features are found in the 
            # variable "remaining_features"
            self.children = np.append(self.children, 
                tree_node(self.data[data_for_feature_value][:,remaining_features],self.labels[data_for_feature_value],
                new_child_diagram_node, self.available_features[remaining_features])) 

    # This function infers (predicts) the class of a new/unseen data point. We call this on the test data points
    def infer(self, data_point):
        if not self.is_leaf: # if the node we are looking at is not a leaf node (can't classify the data point)
            for i in range(self.node_values.shape[0]): # look through the set of values the node looks for
                if self.node_values[i] == data_point[self.feature_index] :   # TODO # to find which branch to descend down
                    allocated_class = self.children[i].infer(data_point[np.arange(data_point.shape[0])!= self.feature_index])  # TODO # recursively run the infer function on the child node (excluding the features which have been used already, see how this was done to get "remaining_features" when decsending the tree above)
                    return allocated_class        # return back up the tree
            print("Error found new value, can't classify")
        else:
            #print("Classified data point as: ", self.class_value)
            return(self.class_value) # If it is a leaf node then we just return the classification given by the leaf node

In [26]:
t = Tree() # Used for diagram, creates tree and adds root node to tree diagram (use as third input to tree_node class)
root = tree_node(train_data[:, 1:],train_data[:, 0],t,features)                # TODO # Use our class to train a decision tree on the training data
print(t.get_ascii(show_internal=True)) # prints a diagram of the decision tree
# The remainder of the window is use to draw the decsision tree. Note the last line can be removed to
# avoid rendering the image as it can look quite bad with large trees. The printed ascii version can still be seen
# in this case
ts = TreeStyle()
ts.show_leaf_name = False
def my_layout(node):
    F = TextFace(node.name, tight_text=True)
    F.rotable = True
    F.border.width = 0
    F.margin_top = 5
    F.margin_bottom = 5
    F.margin_left = 5
    F.margin_right = 5
    add_face_to_node(F, node, column=0, position="branch-right")
ts.layout_fn = my_layout
ts.mode = 'r'
ts.arc_start = 270
ts.arc_span = 185
ts.draw_guiding_lines = True
ts.scale = 100
ts.branch_vertical_margin = 100
ts.min_leaf_separation = 100
ts.show_scale = False
#t.render(file_name="%%inline", w=500, h=500, tree_style=ts)

Working on feature:  0
[5.83590132 0.89049164 0.89049164 0.89049164 0.89049164 0.89049164
 0.89049164 0.89049164 0.89049164]
Working on feature:  1
[5.83590132 6.69204826 0.89049164 0.89049164 0.89049164 0.89049164
 0.89049164 0.89049164 0.89049164]
Working on feature:  2
[5.83590132 6.69204826 4.85086284 0.89049164 0.89049164 0.89049164
 0.89049164 0.89049164 0.89049164]
Working on feature:  3
[5.83590132 6.69204826 4.85086284 6.5323115  0.89049164 0.89049164
 0.89049164 0.89049164 0.89049164]
Working on feature:  4
[5.83590132 6.69204826 4.85086284 6.5323115  7.08769413 0.89049164
 0.89049164 0.89049164 0.89049164]
Working on feature:  5
[5.83590132 6.69204826 4.85086284 6.5323115  7.08769413 6.35330817
 0.89049164 0.89049164 0.89049164]
Working on feature:  6
[5.83590132 6.69204826 4.85086284 6.5323115  7.08769413 6.35330817
 6.79657993 0.89049164 0.89049164]
Working on feature:  7
[5.83590132 6.69204826 4.85086284 6.5323115  7.08769413 6.35330817
 6.79657993 5.81224222 0.89049164]


[3.40217235 2.30160358 0.76420451 0.76420451]
Working on feature:  2
[3.40217235 2.30160358 3.52391148 0.76420451]
Working on feature:  3
[3.40217235 2.30160358 3.52391148 2.60314838]
Found feature:  deg-malig
Descending tree with node entropy value:  0.7642045065086203
Working on feature:  0
[2.05416456 0.76420451 0.76420451]
Working on feature:  1
[2.05416456 1.51474756 0.76420451]
Working on feature:  2
[2.05416456 1.51474756 1.73696978]
Found feature:  age
Descending tree with node entropy value:  0.7642045065086203
Working on feature:  0
[1. 1.]
Working on feature:  1
[1. 1.]
Found feature:  tumor-size
Descending tree with node entropy value:  1.0
Working on feature:  0
[1. 1.]
Working on feature:  1
[1. 1.]
Found feature:  tumor-size
Descending tree with node entropy value:  1.0
Working on feature:  0
[1.]
Found feature:  breast-quad
Descending tree with node entropy value:  1.0
Working on feature:  0
[2.67443004 0.69621226 0.69621226]
Working on feature:  1
[2.67443004 1.6183927

[3.29064673 3.91071321 2.38666621 2.91860475 4.1233284  0.8812909
 0.8812909 ]
Working on feature:  5
[3.29064673 3.91071321 2.38666621 2.91860475 4.1233284  3.35258975
 0.8812909 ]
Working on feature:  6
[3.29064673 3.91071321 2.38666621 2.91860475 4.1233284  3.35258975
 3.98859476]
Found feature:  deg-malig
Descending tree with node entropy value:  0.8812908992306927
Working on feature:  0
[2.78415913 0.97986876 0.97986876 0.97986876 0.97986876 0.97986876]
Working on feature:  1
[2.78415913 2.60568334 0.97986876 0.97986876 0.97986876 0.97986876]
Working on feature:  2
[2.78415913 2.60568334 1.87610938 0.97986876 0.97986876 0.97986876]
Working on feature:  3
[2.78415913 2.60568334 1.87610938 2.47986876 0.97986876 0.97986876]
Working on feature:  4
[2.78415913 2.60568334 1.87610938 2.47986876 2.11400546 0.97986876]
Working on feature:  5
[2.78415913 2.60568334 1.87610938 2.47986876 2.11400546 2.60568334]
Found feature:  age
Descending tree with node entropy value:  0.9798687566511528
W

In [27]:
count_correct = 0
for i in range(train_data.shape[0]):
    out = root.infer(train_data[i, 1:])                   # TODO # infer the class on the data point
    is_correct = (out==train_data[i, 0])          # TODO # check that the chosen class is the same as the true class label
    if is_correct:
        count_correct = count_correct + 1
print("Final Training Accuracy: ", count_correct/train_data.shape[0])
# Note test accuracy should be around 90% cause I've flipped 10% of train data labels

Final Training Accuracy:  0.9773755656108597


In [28]:
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(test_data)):
    out = root.infer(test_data[i, 1:])
    if test_data[i, 0] == out and out=="recurrence-events":
        TP += 1
    elif test_data[i, 0] == out and out=="no-recurrence-events":
        TN += 1
    elif out=="recurrence-events":
        FP += 1
    elif out=="no-recurrence-events":
        FN
accuracy = 'accuracy is: '+str(round(((TP+TN)/(TP+TN+FP+FN))*100))+'%'
d = {'approxiamted reccurence-events': [TP,FP], 'approximated no-reccurence-events': [FN,TN]}
df = pd.DataFrame(data=d,index=['actual reccurence-events','actual no-reccurence-events'])

Error found new value, can't classify
Error found new value, can't classify
Error found new value, can't classify
Error found new value, can't classify
Error found new value, can't classify
Error found new value, can't classify


In [29]:
accuracy

'accuracy is: 80%'

In [30]:
df

,approxiamted reccurence-events,approximated no-reccurence-events
actual reccurence-events,0,0
actual no-reccurence-events,2,8
